In [2]:
import pandas as pd
import ast
import pickle
from annoy import AnnoyIndex
import numpy as np
df = pd.read_csv("embeddings.csv").rename(columns={"bow_embedding_count": "bow_embedding", "bow_embedding_tfidf": "tfidf_embedding"})
# df.head()

,original_title,overview,bow_embedding,tfidf_embedding,glove_embedding,distilbert_embedding
0,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.020005550235509872, 0.21014796197414398, 0...",[-2.50181526e-01 2.13542148e-01 2.42972970e-...
1,Jumanji,When siblings Judy and Peter discover an encha...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.031650930643081665, 0.23956623673439026, 0....",[-2.30265379e-01 1.25701606e-01 2.22994342e-...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.07700946927070618, 0.17933276295661926, 0....",[-7.44484961e-02 -1.16385005e-01 2.24213406e-...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.08693083375692368, 0.11770206689834595, 0.2...",[ 1.23767488e-01 8.40852708e-02 4.18685287e-...
4,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.05611945316195488, 0.0037858595605939627, 0...",[-1.70990497e-01 -9.64889675e-02 4.28808451e-...


In [3]:
def parse_vector_safe(text):
    try:
        if isinstance(text, str):
            cleaned = text.replace('[', '').replace(']', '')
            numbers = [float(x) for x in cleaned.split()]
            return numbers
        elif isinstance(text, list):
            return text 
    except:
        return []

In [4]:
methods = ["glove_embedding", "bow_embedding", "tfidf_embedding", "distilbert_embedding"]

df['glove_embedding'] = df['glove_embedding'].apply(ast.literal_eval)
df["bow_embedding"] = df["bow_embedding"].apply(ast.literal_eval)
df["tfidf_embedding"] = df["tfidf_embedding"].apply(ast.literal_eval)
df["distilbert_embedding"] = df["distilbert_embedding"].apply(parse_vector_safe)


In [7]:
vector_db = {}
for idx, row in df.iterrows():
    vector_db[idx] = {
        "title": row["original_title"],
        "glove": row["glove_embedding"],
        "bow": row["bow_embedding"],
        "tfidf": row["tfidf_embedding"],
        "distilbert": row["distilbert_embedding"]
    }

In [10]:
with open("vector_db.pkl", "wb") as f:
    pickle.dump(vector_db, f)

for method in ["glove", "bow", "tfidf", "distilbert"]:
    dim = len(vector_db[0][method])
    index = AnnoyIndex(dim, 'angular')
    for idx in vector_db:
        index.add_item(idx, vector_db[idx][method])
    index.build(10) 
    index.save(f"{method}_index.ann")

print("Fichiers générés : vector_db.pkl + 4 fichiers .ann")


✅ Fichiers générés : vector_db.pkl + 4 fichiers .ann
